In [1]:
import vk
import pandas as pd
import time
from datetime import datetime

In [2]:
# opennig vk session with access token.
session = vk.Session(access_token='5951a0707374abf660b6aae5bbbc5df47d240d1641765ddd07f681a3de8a8ce7ba1f404c645c58d365a24')
api = vk.API(session)
# example use with id, my id
api.users.get(user_ids='architectofreality' )

[{'first_name': 'Alexander', 'last_name': 'Ovchinnikov', 'uid': 182927367}]

In [3]:
#searching for a keyword glaukoma starting 48 hours ago, max 200 counts can be scrapped at once
glaukoma=api.newsfeed.search(q='glaukoma',count=200, start_time=48)

### get last 200 posts --> take the time of the last post --> at the next iteration set the end time = time_last_post-1. Set the start time to 1 

In [4]:
%%time
glaukoma_rus=api.newsfeed.search(q='glaukoma',count=200, end_time=1512378120, start_time=1512378121-86400)

Wall time: 259 ms


In [5]:
# dropping the first element and creat a dataframe with the rest
df=pd.DataFrame(glaukoma_rus[1:])

In [6]:
df.head(2)

,attachment,attachments,comments,date,from_id,id,likes,marked_as_ads,owner_id,post_source,post_type,reposts,text
0,"{'type': 'photo', 'photo': {'pid': 456239772, ...","[{'type': 'photo', 'photo': {'pid': 456239772,...","{'count': 0, 'can_post': 1}",1512322597,-70740559,1034,"{'count': 0, 'user_likes': 0, 'can_like': 1, '...",0,-70740559,"{'type': 'api', 'platform': 'iphone'}",post,"{'count': 0, 'user_reposted': 0}",Наиболее частым заболеванием хрусталика являет...


In [7]:
df.columns

Index(['attachment', 'attachments', 'comments', 'date', 'from_id', 'id',
       'likes', 'marked_as_ads', 'owner_id', 'post_source', 'post_type',
       'reposts', 'text'],
      dtype='object')

In [8]:
#getting info of each user, deactivated column
ids=pd.DataFrame(api.users.get(user_ids=abs(df['owner_id'])))
ids.head()

,deactivated,first_name,last_name,uid
0,deleted,Elena,Zatylkina,70740559


In [9]:
%%time
desired_time = 86400*30 # how much back we want to go in seconds
date = time.time() # now
excess_time = True
all_df = pd.DataFrame()
c=1
while excess_time:
    print(c) # print iteration count, extended tells us group vs user
    g=api.newsfeed.search(q=['близорукость'],count=200, end_time = date-1, start_time=1,extended=1)
    if len(g)>1: # check if g contains data
        all_df = pd.concat([all_df,pd.DataFrame(g[1:])])
        date=g[-1]['date']
    else:
        time.sleep(0.3)
    c+=1
    if date<time.time()-desired_time: # check termination condition
        excess_time = False

# resetting the index it make cicles of 0-200
all_df.reset_index(inplace=True)

# take all positive ids ( user ids , negative ids are group ids)
all_df_new=all_df[all_df.owner_id>0] 

# get all users
all_id=pd.DataFrame(api.users.get(user_ids=all_df_new['owner_id']))

# fill active users status with valid
all_id.fillna('Valid',inplace=True) 
try:    
    # select all deactivated user ids
    deactivated_ids = list(all_id[all_id['deactivated']!='Valid'].uid)

    # select all active user posts
    active_users = all_df_new[-all_df_new.owner_id.isin(deactivated_ids)] 
except:
    active_users = all_df_new  

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
Wall time: 1min 14s


In [10]:
# creating a dataframe for dictionary values
from tqdm import tqdm
new_df = pd.DataFrame(columns = ['user_id','post','sex','Name_Surname',
                                 'likes','comments','reposts','platform','date'])
active_users.reset_index(inplace = True)
for i in tqdm(range(len(active_users))):    
    new_df.loc[i,'user_id'] = active_users.user.loc[i]['uid']
    new_df.loc[i,'sex'] = active_users.user.loc[i]['sex']   
    new_df.loc[i,'Name_Surname'] = active_users.user.loc[i]['first_name']+'_'+active_users.user.loc[i]['last_name']
    new_df.loc[i,'post'] = active_users.text.loc[i]
    new_df.loc[i,'likes'] = active_users.likes.loc[i]['count']
    new_df.loc[i,'comennts'] = active_users.comments.loc[i]['count'] 
    new_df.loc[i,'reposts'] = active_users.reposts.loc[i]['count']
    try:
        new_df.loc[i,'platform'] = active_users.post_source.loc[i]['type']+'_'+active_users.post_source.loc[i]['platform']
    except:
        new_df.loc[i,'platform'] = active_users.post_source.loc[i]['type']
    new_df.loc[i,'date'] = datetime.fromtimestamp(active_users.date.loc[i]).strftime('%Y-%m-%d %H:%M:%S')
new_df['sex'] = new_df['sex'].apply(lambda x: 'Female' if x==1 else 'Male' if x==2  else 'Not Specified')
new_df

100%|█████████████████████████████████████████████████████████████████████████████| 1305/1305 [00:10<00:00, 127.39it/s]


,user_id,post,sex,Name_Surname,likes,comments,reposts,platform,date,comennts
0,246837046.0,"Где глаза мои были, не знаю, <br>Почему я тебя...",Male,Alexey_Gagin,0.0,NaN,0.0,vk,2017-12-14 15:41:27,0.0
1,15167157.0,Мыгыгы устроено несколько иначе: Обдюсер Ивано...,Male,Ivan_Ivanov,0.0,NaN,0.0,vk,2017-12-14 15:24:29,0.0
2,379795652.0,"Если можно, то мне эспрессо, <br>новостей и не...",Female,Svetlana_Kim,3.0,NaN,0.0,api,2017-12-14 15:10:40,0.0
3,417980972.0,"Будьте Здоровы и осознанны, делюсь полезной ин...",Female,Elena_Afanasyeva,1.0,NaN,0.0,vk,2017-12-14 14:38:02,0.0
4,358722371.0,Дорогие друзья и гости моей странички. На сего...,Female,Olga_Litvinenko,1.0,NaN,0.0,vk,2017-12-14 14:12:24,0.0
5,266044772.0,"Если можно, то мне эспрессо, <br>новостей и не...",Female,Lyubov_Dzyura,0.0,NaN,0.0,api_iphone,2017-12-14 13:51:56,0.0
6,12975755.0,В XIX – начале XX веков наркотические средства...,Male,Andrey_Porokhov,0.0,NaN,0.0,api_wphone,2017-12-14 13:51:48,0.0
7,4847500.0,Джинн трясся от злости и походил на дрожащее о...,Male,Alexander_Lutsenko,0.0,NaN,0.0,vk,2017-12-14 13:48:50,0.0
8,337990091.0,"Если можно, то мне эспрессо,<br>Новостей и нем...",Female,Anna_Gimranova,2.0,NaN,0.0,api_android,2017-12-14 13:36:54,0.0
9,260673450.0,"Если можно, то мне эспрессо, <br>новостей и не...",Female,Natalya_Popova,0.0,NaN,0.0,vk,2017-12-14 12:53:44,0.0


In [11]:
short_df =new_df.copy()

In [12]:
# remove posts with more than 300 characters
short_df['post2']=short_df.post.apply(lambda x: x if len(x)<300 else 'Too Long')

In [13]:
short_df[short_df['post2']!='Too Long']

,user_id,post,sex,Name_Surname,likes,comments,reposts,platform,date,comennts,post2
10,380420908.0,"Ты носишь очки?<br>- да, ты столько лет любишь...",Female,Alsu_Zakharova,2.0,NaN,0.0,mvk,2017-12-14 12:53:33,0.0,"Ты носишь очки?<br>- да, ты столько лет любишь..."
13,244469634.0,,Female,Lyudmila_Vasyuchenkova,0.0,NaN,0.0,vk,2017-12-14 12:12:23,0.0,
24,4016899.0,Врач:<br>— Прочтите нижнюю строчку…<br>Мужик:<...,Male,Vovka_Morkovka,8.0,NaN,0.0,api_android,2017-12-14 00:55:43,0.0,Врач:<br>— Прочтите нижнюю строчку…<br>Мужик:<...
34,233173559.0,"Заказать пластическую хирургию лица, а также в...",Female,Zhansaule_Gisaeva,1.0,NaN,0.0,vk,2017-12-13 22:19:28,0.0,"Заказать пластическую хирургию лица, а также в..."
35,206974791.0,Как восстановить зрение в домашних условиях — ...,Female,Olga_Smirnova,0.0,NaN,0.0,vk,2017-12-13 22:11:40,0.0,Как восстановить зрение в домашних условиях — ...
36,192136990.0,Дамам на заметку.<br>В последнее время часто щ...,Male,Konstantin_Kot,0.0,NaN,0.0,api_android,2017-12-13 22:11:25,0.0,Дамам на заметку.<br>В последнее время часто щ...
51,208277985.0,Лечение близорукости лазером: современные мето...,Female,Natalya_Ivanova,0.0,NaN,0.0,vk,2017-12-13 14:50:57,0.0,Лечение близорукости лазером: современные мето...
52,355109945.0,"""Познай самого себя и ты познаёшь весь мир"" - ...",Female,Lyumika_Stepanyuk,3.0,NaN,0.0,api_iphone,2017-12-13 14:31:35,0.0,"""Познай самого себя и ты познаёшь весь мир"" - ..."
67,152872615.0,Как восстановить зрение в домашних условиях - ...,Female,Viktoria_Saglay,0.0,NaN,0.0,api_android,2017-12-13 10:58:33,0.0,Как восстановить зрение в домашних условиях - ...
73,170454735.0,ПОСЛЕДНЯЯ ЗАКУПКА В ЭТОМ ГОДУ!!!ИДЕТ ОПЛАТА. П...,Female,Aygul_Khabibullina,0.0,NaN,0.0,api,2017-12-13 06:45:06,0.0,ПОСЛЕДНЯЯ ЗАКУПКА В ЭТОМ ГОДУ!!!ИДЕТ ОПЛАТА. П...
